In [ ]:
#recheck dependencies 

In [22]:
import panel as pn
import numpy as np
from pycromanager import Core
from bokeh.plotting import figure
from bokeh.events import Tap, SelectionGeometry
from bokeh.models import BoxEditTool, ColumnDataSource

pn.extension() #render panel inline

In [2]:
core = Core()

# Display and hist tab

In [74]:
#widgets
image_button = pn.widgets.Button(name='Snap Image', button_type='primary') #panel widget
image_widget = pn.panel(figure(title='Display')) #bokeh wrapper
histo_button = pn.widgets.Button(name='Histogram', button_type='primary')
histo_widget = pn.panel(figure(title='Histogram'))

up_button    = pn.widgets.Button(name='^', button_type='primary')
left_button  = pn.widgets.Button(name='<', button_type='primary')
right_button = pn.widgets.Button(name='>', button_type='primary')
down_button  = pn.widgets.Button(name='v', button_type='primary')
coord_text   = pn.widgets.TextInput(value=str([core.get_x_position(),core.get_y_position()]))
step_size    = pn.widgets.FloatInput(name='step size', value=1., step=1e-1, start=0, end=1500)

xy_stage=core.get_xy_stage_device() #set stage to move

#MM snap image
def snap():
    core.wait_for_system()
    core.snap_image()
    tagged_image = core.get_tagged_image()
    
    return tagged_image


#on_click callbacks
def on_image_button_clicked(b):
    core.clear_roi()
    tagged_image=snap()
    
    #reshape to 3d array
    height = tagged_image.tags['Height']
    width = tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix,
                    newshape=[height, width])#, tagged_image.pix.shape[0]//(height*width)])
    
    #reshape to 2d array for bokeh's image_rgba
    #image = np.flipud(pixels.view("uint32"))#.reshape(pixels.shape[:2]))
    
    
    resize = 500/pixels.shape[1] #resize to have widht = 500
    
    #create blank figure for image
    #set tooltips for displaying hover (x and y coord, image value)
    p=figure(title= 'Display',tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], 
             frame_width=500, frame_height=int(pixels.shape[0]*resize))
    p.image(image=[pixels],x=0, y=0, dw=pixels.shape[0], dh=pixels.shape[1])
    image_widget.object=p #update widget
                                             

def on_histo_button_clicked(b):
    tagged_image= snap()
    
    # Histogram
    hist, edges = np.histogram(tagged_image.pix, density=True, bins=64)
    y_max=max(hist) + (max(hist)/10) #give room at the top
    #set tooltips to dispaly bokeh's quad's "top" input
    p=figure(title='Histogram',tooltips=[("Value", "@top")], y_range=(0, y_max), frame_width=1280)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="lightgrey", line_color="black")
    histo_widget.object=p #update widget
    
def on_move_up_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_left_button_clicked(b):
    core.set_relative_xy_position(xy_stage, -step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])

def on_move_right_button_clicked(b):
    core.set_relative_xy_position(xy_stage, step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_down_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,-step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
        

#on_click events
image_button.on_click(on_image_button_clicked)
histo_button.on_click(on_histo_button_clicked)
up_button.on_click(on_move_up_button_clicked)
left_button.on_click(on_move_left_button_clicked) 
right_button.on_click(on_move_right_button_clicked)
down_button.on_click(on_move_down_button_clicked)


#dash layout
display_app = pn.Column(
    pn.Row(
        pn.Column(
            image_button, 
            histo_button,
            step_size,
            pn.Row(up_button,align='center'),
            pn.Row(left_button, right_button, align='center'),
            pn.Row(down_button, align='center'),
            coord_text, sizing_mode='stretch_height'),
        image_widget),
    histo_widget)


# ROI tab

In [70]:
#widgets
display_button = pn.widgets.Button(name='Display', button_type='primary') #panel widget
display_widget = pn.panel(figure(name='Display')) #bokeh wrapper


roi_button   = pn.widgets.Button(name='ROI', button_type='primary')
roi_widget   = pn.panel(figure())

#MM snap image
def snap():
    core.wait_for_system()
    core.snap_image()
    tagged_image = core.get_tagged_image()
    
    return tagged_image

box_coord=[]

def box(event):
    box_coord.clear()
    box_coord.append(event.geometry)

    
#on_click callbacks
def on_display_button_clicked(b):
    core.clear_roi()
    tagged_image=snap()
    
    #reshape to 3d array
    height = tagged_image.tags['Height']
    width = tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix,
                    newshape=[height, width])#, tagged_image.pix.shape[0]//(height*width)])
    
    #reshape to 2d array for bokeh's image_rgba
    #image = np.flipud(pixels.view("uint32"))#.reshape(pixels.shape[:2]))
    
    
    resize = 500/pixels.shape[1] #resize to have widht = 500
    
    #create blank figure for image
    #set tooltips for displaying hover (x and y coord, image value)
    p=figure(title= 'Display',tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], 
             frame_width=500, frame_height=int(pixels.shape[0]*resize))
    p.image(image=[pixels],x=0, y=0, dw=pixels.shape[0], dh=pixels.shape[1])
    image_widget.object=p #update widget

def on_roi_button_clicked(b):
    if len(box_coord) != 0:
        
        x=int(box_coord[0]['x0'])
        y=int(box_coord[0]['y0'])
        dx=int(abs(x-box_coord[0]['x1']))
        dy=int(abs(y-box_coord[0]['y1']))
        
        core.clear_roi()
        core.set_roi(x,y,dx,dy)
        
        #display image
        tagged_image=snap()
    
        #reshape to 3d array
        height = tagged_image.tags['Height']
        width = tagged_image.tags['Width']
        pixels = np.reshape(tagged_image.pix,
                        newshape=[height, width])#, tagged_image.pix.shape[0]//(height*width)])

        #reshape to 2d array for bokeh's image_rgba
        #image = np.flipud(pixels.view("uint32"))#.reshape(pixels.shape[:2]))
        #create blank figure for image
        #set tooltips for displaying hover (x and y coord, image value)
        p=figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], frame_width=500, 
                 frame_height=int(pixels.shape[0]*(500/pixels.shape[1])))

        p.image(image=[pixels],x=0, y=0, dw=pixels.shape[0], dh=pixels.shape[1])
        roi_widget.object=p #update widget
        
display_button.on_click(on_display_button_clicked)
roi_button.on_click(on_roi_button_clicked)

roi_app = pn.Row(pn.Column(display_button, display_widget), pn.Column(roi_button, roi_widget))

In [65]:
20*64

1280

# swabian tab

In [5]:
import TimeTagger as TT
from TimeTagger import Flim, TimeTagStream, createTimeTaggerVirtual, EventGenerator, Countrate,ConstantFractionDiscriminator, DelayedChannel
from bokeh.models.mappers import LogColorMapper
from bokeh.models import ColorBar, LogTicker

In [6]:
dump_file = r"Z:\PriyaM\swabian_data\20230712_tissue\test_tissue_2023-07-12_154752.1.ttbin"
time_tagger= TT.createTimeTaggerVirtual()


In [19]:
#check ranges?
laser_num_widg = pn.widgets.IntInput(name='Laser Channel Num', value=1, step=1, start=-10, end=10)
laser = laser_num_widg.value
line_num_widg  = pn.widgets.IntInput(name='Line Channel Num', value=2, step=1, start=-10, end=10)
line = line_num_widg.value
frame_num_widg = pn.widgets.IntInput(name='Frame Channel Num', value=3, step=1, start=-10, end=10)
frame = frame_num_widg.value
photon_num_widg = pn.widgets.IntInput(name='Photon Channel Num', value=-4, step=1, start=-10, end=10)
photon = photon_num_widg.value

n_bins_widg   = pn.widgets.IntInput(name='Number of Bins', value=256, step=1, start=0, end=2048)
#check if bw can be calc?
binwidth_widg = pn.widgets.IntInput(name='Binwidth', value=49, step=1, start=0, end=1000)
#can this just be n_bins?
n_pixel_widg  = pn.widgets.IntInput(name='Number of Pixels', value=256, step=1, start=0, end=2048)
n_pixel = n_pixel_widg.value
#do these need to be set?
pixel_rate_widg = pn.widgets.FloatInput(name='Number of Pixels', value=200e3, step=1e3, start=0, end=400e3)
pixel_time = 1/pixel_rate_widg.value
integ_time_widg = pn.widgets.FloatInput(name='Number of Pixels', value=3e9, step=1e9, start=0, end=10e9)


flim_button = pn.widgets.Button(name='Start Flim Measurement', button_type='primary') #panel widget
flim_widget = pn.panel(figure()) #bokeh wrapper

# EventGenerator(tagger, trigger_channel, pattern, trigger_divider, stop_channel)
#virtual channel
pixel_pattern_start = np.linspace(0, n_pixel, num=n_pixel, endpoint=False, dtype=int) * (pixel_time * 1e12)
pixel_pattern_end = pixel_pattern_start + integ_time_widg.value

pixel_start_chan = TT.EventGenerator(time_tagger, line, pixel_pattern_start)
pixel_end_chan= TT.EventGenerator(time_tagger, line, pixel_pattern_end)

#ConstantFractionDiscriminator(tagger, channels, search_window)
#virtual channel
photon_cfd_chan = TT.ConstantFractionDiscriminator(time_tagger, (photon,), 10*1000)

#DelayedChannel(tagger, input_channel, delay)
#virtual channel
delayed_laser_chan = TT.DelayedChannel(time_tagger, laser, 11.5e3)

color_mapper = LogColorMapper(palette="Viridis256", low=1, high=20)

def on_flim_button_clicked(b):
    flim = TT.Flim(time_tagger, start_channel=delayed_laser_chan.getChannel(), click_channel=photon_cfd_chan.getChannels()[0], pixel_begin_channel=pixel_start_chan.getChannel(), 
            n_pixels=n_pixel*n_pixel, n_bins=n_bins_widg.value, binwidth=binwidth_widg.value, pixel_end_channel=pixel_end_chan.getChannel(), frame_begin_channel=frame)
    time_tagger.setReplaySpeed(-1)
    replay = time_tagger.replay(dump_file)
    time_tagger.waitForCompletion()
    flim.getFramesAcquired()

    flim_frames = flim.getSummedFrames()
    flim_2d = flim_frames.reshape(256,256,256)
    image = flim_2d.sum(2)

    t=figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],frame_width=image.shape[1]*2, frame_height=image.shape[0]*2)
    t.image(image=[image],x=0, y=0, dw=image.shape[0], dh=image.shape[1],color_mapper=color_mapper)
    color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))
    t.add_layout(color_bar, 'right')
    flim_widget.object=t #update widget
    
#on_click events
flim_button.on_click(on_flim_button_clicked)

settings = pn.Column(
    laser_num_widg,
    line_num_widg,
    frame_num_widg,
    photon_num_widg,
    n_bins_widg,  
    binwidth_widg,
    n_pixel_widg,  
    pixel_rate_widg, 
    integ_time_widg
)

flim_app = pn.Row(settings,pn.Column(flim_button, flim_widget))

# tabs

In [75]:
tabs = pn.Tabs(('Display Image',display_app), ('ROI',roi_app), ('Flim',flim_app), dynamic=True)
tabs.servable() #must call servable() for it to render in server

Tabs(dynamic=True)
    [0] Column
        [0] Row
            [0] Column(sizing_mode='stretch_height')
                [0] Button(button_type='primary', name='Snap Image')
                [1] Button(button_type='primary', name='Histogram')
                [2] FloatInput(end=1500, name='step size', start=0, value=1.0)
                [3] Row(align='center')
                    [0] Button(button_type='primary', name='^')
                [4] Row(align='center')
                    [0] Button(button_type='primary', name='<')
                    [1] Button(button_type='primary', name='>')
                [5] Row(align='center')
                    [0] Button(button_type='primary', name='v')
                [6] TextInput(value='[0, 0]')
            [1] Bokeh(figure)
        [1] Bokeh(figure)
    [1] Row
        [0] Column
            [0] Button(button_type='primary', name='Display')
            [1] Bokeh(figure)
        [1] Column
            [0] Button(button_type='primary', name='ROI')
            [1] Bokeh(figure)
    [2] Row
        [0] Column
            [0] IntInput(end=10, name='Laser Channel Num', start=-10, value=1)
            [1] IntInput(end=10, name='Line Channel Num', start=-10, value=2)
            [2] IntInput(end=10, name='Frame Channel Num', start=-10, value=3)
            [3] IntInput(end=10, name='Photon Channel Num', start=-10, value=-4)
            [4] IntInput(end=2048, name='Number of Bins', start=0, value=256)
            [5] IntInput(end=1000, name='Binwidth', start=0, value=49)
            [6] IntInput(end=2048, name='Number of Pixels', start=0, value=256)
            [7] FloatInput(end=400000.0, name='Number of Pixels', start=0, step=1000.0, value=200000.0)
            [8] FloatInput(end=10000000000.0, name='Number of Pixels', start=0, step=1000000000.0, value=3000000000.0)
        [1] Column
            [0] Button(button_type='primary', clicks=1, name='Start Flim Measurement')
            [1] Bokeh(figure, align='auto', height=1000, width=1000)